# 📊 מדריך מקיף לניתוח סדרות זמן עם Polars

## פרק 9: Time Series Analysis + Forecasting

---

### 👋 ברוכים הבאים!

מדריך זה יעזור לכם ללמוד כיצד לעבוד עם סדרות זמן (Time Series) באמצעות ספריית **Polars**.

### 🎯 מה נלמד במדריך זה?

1. **עבודה עם תאריכים ושעות** - המרה, פירוק, וסינון
2. **חלונות מתגלגלים (Rolling Windows)** - חישוב ממוצעים נעים
3. **דגימת יתר ותת-דגימה (Resampling)** - שינוי רזולוציית הזמן
4. **חיזוי סדרות זמן (Forecasting)** - חיזוי עם Functime 🔮

### 📚 תוכן עניינים

- [1. הכנה - טעינת נתונים](#section-1)
- [2. עבודה עם תאריכים ושעות](#section-2)
- [3. חלונות מתגלגלים (Rolling Windows)](#section-3)
- [4. טכניקות דגימה מחדש (Resampling)](#section-4)
- [5. חיזוי סדרות זמן (Forecasting)](#section-5)
- [6. תרגילים אינטראקטיביים](#section-6)
- [7. סיכום וטיפים](#section-7)

---

<a id='section-1'></a>
## 1. 🚀 הכנה - טעינת נתונים

נתחיל בטעינת קובץ CSV שמכיל נתוני מזג אוויר מטורונטו.

In [12]:
# ייבוא הספרייה
import polars as pl

# טעינת הנתונים כ-LazyFrame
lf = pl.scan_csv('../data/toronto_weather.csv')

# הצגת 5 השורות הראשונות
lf.head().collect()

datetime,temperature,wind_speed,pressure,humidity
str,f64,f64,f64,f64
"""2012-10-01 12:00:00""",null,null,null,null
"""2012-10-01 13:00:00""",286.26,3.0,1012.0,82.0
"""2012-10-01 14:00:00""",286.262541,3.0,1011.0,81.0
"""2012-10-01 15:00:00""",286.269518,3.0,1011.0,79.0
"""2012-10-01 16:00:00""",286.276496,3.0,1010.0,77.0


### 🌡️ המרת טמפרטורה מקלווין לצלזיוס

In [13]:
# המרה לצלזיוס
lf = lf.with_columns(
    (pl.col('temperature') - 273.15).round(2).alias('temperature')
)

# בדיקת התוצאה
lf.head().collect()

datetime,temperature,wind_speed,pressure,humidity
str,f64,f64,f64,f64
"""2012-10-01 12:00:00""",null,null,null,null
"""2012-10-01 13:00:00""",13.11,3.0,1012.0,82.0
"""2012-10-01 14:00:00""",13.11,3.0,1011.0,81.0
"""2012-10-01 15:00:00""",13.12,3.0,1011.0,79.0
"""2012-10-01 16:00:00""",13.13,3.0,1010.0,77.0


---
<a id='section-2'></a>
## 2. 📅 עבודה עם תאריכים ושעות

### ניתוח תאריכים אוטומטי

In [14]:
# טעינה עם ניתוח תאריכים אוטומטי
lf_date_parsed = pl.scan_csv('../data/toronto_weather.csv', try_parse_dates=True)

# הצגת הנתונים
lf_date_parsed.head().collect()

datetime,temperature,wind_speed,pressure,humidity
datetime[μs],f64,f64,f64,f64
2012-10-01 12:00:00,null,null,null,null
2012-10-01 13:00:00,286.26,3.0,1012.0,82.0
2012-10-01 14:00:00,286.262541,3.0,1011.0,81.0
2012-10-01 15:00:00,286.269518,3.0,1011.0,79.0
2012-10-01 16:00:00,286.276496,3.0,1010.0,77.0


### המרת תאריכים ידנית

In [15]:
# המרה ידנית של עמודת datetime
lf = lf.with_columns(
    pl.col('datetime').str.to_datetime()
)

lf.head().collect()

datetime,temperature,wind_speed,pressure,humidity
datetime[μs],f64,f64,f64,f64
2012-10-01 12:00:00,null,null,null,null
2012-10-01 13:00:00,13.11,3.0,1012.0,82.0
2012-10-01 14:00:00,13.11,3.0,1011.0,81.0
2012-10-01 15:00:00,13.12,3.0,1011.0,79.0
2012-10-01 16:00:00,13.13,3.0,1010.0,77.0


### פירוק תאריכים לרכיבים

In [16]:
# פירוק התאריך לרכיבים
(
    lf
    .select(
        'datetime',
        pl.col('datetime').dt.year().alias('year'),
        pl.col('datetime').dt.month().alias('month'),
        pl.col('datetime').dt.day().alias('day'),
        pl.col('datetime').dt.time().alias('time')
    )
    .head().collect()
)

datetime,year,month,day,time
datetime[μs],i32,i8,i8,time
2012-10-01 12:00:00,2012,10,1,12:00:00
2012-10-01 13:00:00,2012,10,1,13:00:00
2012-10-01 14:00:00,2012,10,1,14:00:00
2012-10-01 15:00:00,2012,10,1,15:00:00
2012-10-01 16:00:00,2012,10,1,16:00:00


### סינון לפי תאריכים

In [17]:
from datetime import datetime

# סינון לשנת 2017, רק בשעות הבוקר (עד 12:00)
filtered_lf = (
    lf
    .filter(
        pl.col('datetime').dt.date().is_between(
            datetime(2017, 1, 1),
            datetime(2017, 12, 31)
        ),
        pl.col('datetime').dt.hour() < 12
    )
)

filtered_lf.head().collect()

datetime,temperature,wind_speed,pressure,humidity
datetime[μs],f64,f64,f64,f64
2017-01-01 00:00:00,2.44,3.0,1001.0,92.0
2017-01-01 01:00:00,2.19,9.0,1003.0,86.0
2017-01-01 02:00:00,2.41,7.0,1003.0,86.0
2017-01-01 03:00:00,2.42,5.0,1003.0,74.0
2017-01-01 04:00:00,1.77,6.0,1006.0,69.0


---
<a id='section-3'></a>
## 3. 📈 חלונות מתגלגלים (Rolling Windows)

### ממוצע נע פשוט

In [18]:
# ממוצע נע של 3 שעות על הטמפרטורה
(
    lf
    .select(
        'datetime',
        'temperature',
        pl.col('temperature').rolling_mean(3).alias('3hr_rolling_avg')
    )
    .head()
    .collect()
)

datetime,temperature,3hr_rolling_avg
datetime[μs],f64,f64
2012-10-01 12:00:00,null,null
2012-10-01 13:00:00,13.11,null
2012-10-01 14:00:00,13.11,null
2012-10-01 15:00:00,13.12,13.113333
2012-10-01 16:00:00,13.13,13.12


### ממוצעים יומיים

In [19]:
# חישוב ממוצע טמפרטורה יומי
daily_avg_temperature_lf = (
    lf
    .select(
        pl.col('datetime').dt.date().alias('date'),
        'temperature'
    )
    .group_by('date', maintain_order=True)
    .agg(
        pl.col('temperature').mean().alias('daily_avg_temp')
    )
)

daily_avg_temperature_lf.head().collect()

date,daily_avg_temp
date,f64
2012-10-01,13.140909
2012-10-02,14.2475
2012-10-03,14.177083
2012-10-04,15.067917
2012-10-05,16.216667


### סטטיסטיקות מתגלגלות

In [20]:
# חישוב ממוצע, מינימום ומקסימום נעים של 3 ימים
(
    daily_avg_temperature_lf
    .select(
        'date',
        'daily_avg_temp',
        pl.col('daily_avg_temp').rolling_mean(3).alias('3day_rolling_avg'),
        pl.col('daily_avg_temp').rolling_min(3).alias('3day_rolling_min'),
        pl.col('daily_avg_temp').rolling_max(3).alias('3day_rolling_max')
    )
    .head()
    .collect()
)

date,daily_avg_temp,3day_rolling_avg,3day_rolling_min,3day_rolling_max
date,f64,f64,f64,f64
2012-10-01,13.140909,null,null,null
2012-10-02,14.2475,null,null,null
2012-10-03,14.177083,13.855164,13.140909,14.2475
2012-10-04,15.067917,14.4975,14.177083,15.067917
2012-10-05,16.216667,15.153889,14.177083,16.216667


---
<a id='section-4'></a>
## 4. ⏱️ טכניקות דגימה מחדש (Resampling)

### תת-דגימה (Downsampling)

In [21]:
# דגימה מחדש לשבועות
(
    lf
    .set_sorted('datetime')
    .group_by_dynamic(
        'datetime',
        every='1w'
    )
    .agg(
        pl.col('humidity').mean().round(1)
    )
    .head(10)
    .collect()
)

datetime,humidity
datetime[μs],f64
2012-10-01 00:00:00,63.1
2012-10-08 00:00:00,62.1
2012-10-15 00:00:00,76.1
2012-10-22 00:00:00,70.0
2012-10-29 00:00:00,80.0
2012-11-05 00:00:00,68.7
2012-11-12 00:00:00,68.5
2012-11-19 00:00:00,81.0
2012-11-26 00:00:00,69.0


### דגימת יתר (Upsampling)

In [22]:
# דגימת יתר - נתונים כל 30 דקות
upsampled_df = (
    lf
    .set_sorted('datetime')
    .collect()
    .upsample(
        time_column='datetime',
        every='30m',
        maintain_order=True
    )
    .select(
        'datetime',
        pl.col('humidity')
    )
)

upsampled_df.head(10)

datetime,humidity
datetime[μs],f64
2012-10-01 12:00:00,null
2012-10-01 12:30:00,null
2012-10-01 13:00:00,82.0
2012-10-01 13:30:00,null
2012-10-01 14:00:00,81.0
2012-10-01 14:30:00,null
2012-10-01 15:00:00,79.0
2012-10-01 15:30:00,null
2012-10-01 16:00:00,77.0


### אינטרפולציה

In [23]:
# מילוי הערכים החסרים עם אינטרפולציה
(
    upsampled_df
    .with_columns(
        pl.col('humidity').interpolate()
    )
    .head(10)
)

datetime,humidity
datetime[μs],f64
2012-10-01 12:00:00,null
2012-10-01 12:30:00,null
2012-10-01 13:00:00,82.0
2012-10-01 13:30:00,81.5
2012-10-01 14:00:00,81.0
2012-10-01 14:30:00,80.0
2012-10-01 15:00:00,79.0
2012-10-01 15:30:00,78.0
2012-10-01 16:00:00,77.0


---
<a id='section-5'></a>
## 5. 🔮 חיזוי סדרות זמן עם Functime

### מהו Time Series Forecasting?

**חיזוי סדרות זמן** הוא תהליך של חיזוי ערכים עתידיים על בסיס נתונים היסטוריים.

### מה זה Functime?

**Functime** היא ספרייה מתקדמת לחיזוי סדרות זמן שפותחה במיוחד לעבוד עם **Polars**.

**יתרונות:**
- ⚡ **מהירות** - אופטימיזציה מלאה ל-Polars
- 🎯 **פשטות** - API נקי וקל לשימוש
- 🔧 **גמישות** - מגוון מודלים וטכניקות

### 🚀 הכנת הנתונים

In [25]:
# טעינת נתוני טמפרטורות היסטוריות
lf_hist = pl.scan_csv('../data/historical_temperatures.csv', try_parse_dates=True)

# הצגת דוגמה
lf_hist.head().collect()

datetime,city,temperature
datetime[μs],str,f64
2012-10-01 12:00:00,"""Toronto""",null
2012-10-01 13:00:00,"""Toronto""",286.26
2012-10-01 14:00:00,"""Toronto""",286.262541
2012-10-01 15:00:00,"""Toronto""",286.269518
2012-10-01 16:00:00,"""Toronto""",286.276496


In [26]:
# רשימת הערים
lf_hist.select('city').unique().sort('city').collect()

city
str
"""Las Vegas"""
"""New York"""
"""San Francisco"""
"""Seattle"""
"""Toronto"""
"""Vancouver"""


In [27]:
# הצגת 3 שורות ראשונות לכל עיר
lf_hist.group_by('city').head(3).collect()

city,datetime,temperature
str,datetime[μs],f64
"""San Francisco""",2012-10-01 12:00:00,null
"""San Francisco""",2012-10-01 13:00:00,289.48
"""San Francisco""",2012-10-01 14:00:00,289.474993
"""New York""",2012-10-01 12:00:00,null
"""New York""",2012-10-01 13:00:00,288.22
…,…,…
"""Seattle""",2012-10-01 13:00:00,281.8
"""Seattle""",2012-10-01 14:00:00,281.797217
"""Toronto""",2012-10-01 12:00:00,null


### 📊 צבירה חודשית

In [28]:
# קבלת שמות העמודות
time_col, entity_col, value_col = lf_hist.collect_schema().names()
print(f"עמודות: {time_col}, {entity_col}, {value_col}")

# צבירה חודשית + המרה לצלזיוס
y = (
    lf_hist
    .group_by_dynamic(
        time_col,
        every='1mo',
        group_by=entity_col,
    )
    .agg(
        (pl.col('temperature').mean() - 273.15).round(1),
    )
)

y.group_by('city').head(3).collect()

עמודות: datetime, city, temperature


city,datetime,temperature
str,datetime[μs],f64
"""Las Vegas""",2012-10-01 00:00:00,20.9
"""Las Vegas""",2012-11-01 00:00:00,14.4
"""Las Vegas""",2012-12-01 00:00:00,9.0
"""San Francisco""",2012-10-01 00:00:00,16.5
"""San Francisco""",2012-11-01 00:00:00,13.4
…,…,…
"""New York""",2012-11-01 00:00:00,5.9
"""New York""",2012-12-01 00:00:00,4.7
"""Seattle""",2012-10-01 00:00:00,11.2


### 🎯 פיצול לנתוני אימון ובדיקה

In [ ]:
from functime.cross_validation import train_test_split

# פיצול
test_size = 3
X = y.select(entity_col, time_col)
y_train, y_test = (
    y
    .select(entity_col, time_col, value_col)
    .pipe(train_test_split(test_size))
)
X_train, X_test = X.pipe(train_test_split(test_size))

print(f"Train: {y_train.collect().shape[0]} שורות")
print(f"Test: {y_test.collect().shape[0]} שורות")

### 🤖 אימון מודל

In [3]:
from functime.forecasting import linear_model

# יצירת המודל
forecaster = linear_model(lags=24, freq='1mo')

# אימון
forecaster.fit(y=y_train)

# חיזוי
y_pred = forecaster.predict(fh=test_size)

print("חיזוי הושלם!")
y_pred.head().collect()

ModuleNotFoundError: No module named 'functime'

### 📊 הערכת דיוק

In [ ]:
from functime.metrics import mase

# חישוב דיוק
scores = mase(
    y_true=y_test,
    y_pred=y_pred,
    y_train=y_train
)

print("ציוני MASE:")
scores.collect()

### 📈 ויזואליזציה

In [ ]:
# גרף הנתונים האמיתיים
actual_viz = (
    y.collect()
    .plot.line(
        x='datetime',
        y='temperature',
        by='city',
        subplots=True
    )
    .cols(2)
)

# גרף החיזויים
pred_viz = (
    y_pred
    .plot.line(
        x='datetime',
        y='temperature',
        by='city',
        subplots=True
    )
    .cols(2)
)

# שילוב
actual_viz * pred_viz

### 🔧 Feature Engineering מתקדם

In [ ]:
from functime.seasonality import add_calendar_effects

# יצירת תכונות מתקדמות
y_features = (
    lf_hist
    .group_by_dynamic(
        time_col,
        every='1mo',
        group_by=entity_col,
    )
    .agg(
        (pl.col('temperature').mean() - 273.15).round(1),
        pl.col(value_col).ts.binned_entropy(bin_count=10).alias('binned_entropy'),
        pl.col(value_col).ts.lempel_ziv_complexity(threshold=3).alias('lempel_ziv_complexity'),
        pl.col(value_col).ts.longest_streak_above_mean().alias('longest_streak_above_mean')
    )
    .pipe(add_calendar_effects(['month']))
)

y_features.head().collect()

### 📝 סיכום - חיזוי

**למדנו:**
- ✅ מהו חיזוי סדרות זמן
- ✅ הכנת נתונים לחיזוי
- ✅ פיצול כרונולוגי train/test
- ✅ אימון מודל Linear עם Lags
- ✅ הערכת דיוק עם MASE
- ✅ ויזואליזציה
- ✅ Feature Engineering

**💡 מודלים נוספים:**
```python
from functime.forecasting import (
    linear_model,
    lightgbm,  # מומלץ!
    xgboost,
    knn,
    catboost
)
```

---
<a id='section-6'></a>
## 6. 🎓 תרגילים אינטראקטיביים

### תרגיל 1: סינון ופירוק תאריכים ⭐

**משימה:** סנן לשנת 2015, הוסף עמודת יום בשבוע, וחשב ממוצע לכל יום.

In [ ]:
# הקוד שלך כאן:



### תרגיל 2: ממוצע נע ⭐⭐

**משימה:** חשב ממוצעים נעים של 7 ו-30 ימים וצור גרף.

In [ ]:
# הקוד שלך כאן:



### תרגיל 3: חיזוי עם LightGBM ⭐⭐⭐⭐

**משימה:** אמן מודל LightGBM במקום Linear וחזה 6 חודשים.

In [ ]:
# הקוד שלך כאן:
# from functime.forecasting import lightgbm



---
<a id='section-7'></a>
## 7. 📚 סיכום וטיפים

### 🎯 מה למדנו?

#### 1. עבודה עם תאריכים
- ✅ טעינה אוטומטית עם `try_parse_dates`
- ✅ פירוק לרכיבים
- ✅ סינון לפי טווחים

#### 2. חלונות מתגלגלים
- ✅ `rolling_mean`, `rolling_min`, `rolling_max`
- ✅ שיטות שונות
- ✅ ויזואליזציה

#### 3. Resampling
- ✅ Downsampling עם `group_by_dynamic`
- ✅ Upsampling עם `upsample`
- ✅ אינטרפולציה

#### 4. חיזוי 🔮
- ✅ אימון מודלים
- ✅ הערכת דיוק
- ✅ Feature Engineering

### 💡 10 טיפים חשובים

1. **תמיד בדוק סוגי נתונים**
2. **השתמש ב-set_sorted()** לפני פעולות זמן
3. **LazyFrame vs DataFrame** - דע מתי להשתמש
4. **בדוק עם head() קודם**
5. **prefer built-in functions**
6. **ויזואליזציה** - תמיד
7. **maintain_order=True** בgroup_by
8. **min_periods=1** למניעת nulls
9. **פצל כרונולוגית** בחיזוי
10. **תעד את הקוד** שלך

### 📖 משאבים נוספים

- **Polars:** https://pola-rs.github.io/polars/
- **Functime:** https://docs.functime.ai/
- **Discord:** https://discord.gg/4UfP5cfBE7

---

## 🎉 סיימתם!

כל הכבוד! עכשיו אתם יודעים לעבוד עם סדרות זמן ב-Polars **ואפילו לחזות את העתיד**! 🔮

**בהצלחה!** 🌟